# AutoRec cs3639 Recommendation Systems course IDC

### here will be general explanations

In [13]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## datasets

In this project, we will use 2 datasets:
* **movielens**, which can be downloaded using `utils.datasets_download.py` or straight from [here](http://files.grouplens.org/datasets/movielens/).
* **netflixprize**, which can be downloaded from this [semi-parsed version from kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data) or from this [raw version](https://archive.org/download/nf_prize_dataset.tar)

**NOTE**: for the notebook to run properly, you should save you dataset under `data` folder and `movielens` folder for the movielens dataset and `netflix` folder for the netflixprize dataset.
i.e `data/movielens` folder and `data/netflix` folder respectively.

In [15]:
from src.data_prep import movielens_load
train, test = movielens_load(1)
print(train.shape)
train

(80000, 4)


,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [ ]:
# TODO:
# - Add dataloder class
# - Add batch size
# - Add bias for the MatrixFactorization

In [20]:
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, k=15):
        super().__init__()
        self.k = k
        self.num_users = num_users
        self.num_items = num_items

        # create the latent matrixs
        self.users_emb = nn.Embedding(num_users, k)
        self.items_emb = nn.Embedding(num_items, k)

        self.batch_size = 100


    def forward(self, user, item):
        user = self.users_emb(user)
        item = self.items_emb(item)
        return (user*item).sum(axis=1)



In [ ]:
# TODO:
# - add function that will calculate the validation loss

In [50]:
def train_epocs(train, model, epochs=10, lr=0.001, reg=0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=reg)
    for i in range(epochs):
        model.train()
        # users and items indexes start 1, therefore we use the -1
        users = torch.LongTensor(train.user_id.values-1)#.to(device)
        items = torch.LongTensor(train.item_id.values-1)#.to(device)
        ratings = torch.FloatTensor(train.rating.values)#.to(device) # rating is our label

        preds = model(users, items)
        train_loss = torch.sqrt(nn.functional.mse_loss(preds, ratings))

        # backpropagation
        optimizer.zero_grad()
        loss.backward()

        # update
        optimizer.step()


        print(f'train RMSE: {train_loss.item()}')


In [51]:
num_users = train.user_id.max()
num_items = train.item_id.max()
print(num_users, num_items)
model = MatrixFactorization(num_users, num_items)#.to(device)


943 1682


In [52]:
train_epocs(train, model, epochs=500, lr=0.001, reg=0.001)

train RMSE: 5.407269477844238
train RMSE: 5.402543067932129
train RMSE: 5.397825241088867
train RMSE: 5.393117427825928
train RMSE: 5.388420581817627
train RMSE: 5.383732795715332
train RMSE: 5.379055976867676
train RMSE: 5.3743896484375
train RMSE: 5.3697333335876465
train RMSE: 5.365087985992432
train RMSE: 5.360452175140381
train RMSE: 5.355827331542969
train RMSE: 5.351212978363037
train RMSE: 5.346609115600586
train RMSE: 5.342016696929932
train RMSE: 5.3374342918396
train RMSE: 5.3328633308410645
train RMSE: 5.328303337097168
train RMSE: 5.323753833770752
train RMSE: 5.319215774536133
train RMSE: 5.314688205718994
train RMSE: 5.3101725578308105
train RMSE: 5.305666923522949
train RMSE: 5.301173686981201
train RMSE: 5.296690940856934
train RMSE: 5.292220592498779
train RMSE: 5.287761211395264
train RMSE: 5.283312797546387
train RMSE: 5.278876304626465
train RMSE: 5.27445125579834
train RMSE: 5.270038604736328
train RMSE: 5.265636444091797
train RMSE: 5.261246204376221
train RMSE: 